In [7]:
import pandas as pd
from sqlalchemy import create_engine
import pyodbc
import psycopg2

In [17]:
conn = psycopg2.connect(
        database="qs",
        user="elton",
        password="Y8ze1eFaSWltR1zl43fr",
        host="172.30.180.232",
        port= '5432'
        )

cursor = conn.cursor()
query = '''
    SELECT asset_data.*, header.*
    FROM acq_msci_barra.risk_report_asset_position_data AS asset_data
    LEFT JOIN acq_msci_barra.risk_report_asset_position_header AS header
    ON asset_data.report_id = header.report_id
'''
df = pd.read_sql(query, conn)
print(df)

                             report_id  indent  barraid       localid  \
0     1cec6fdccdce4740a1d91408aae90ffd       0                          
1     1cec6fdccdce4740a1d91408aae90ffd       1     None  CN601360_CFD   
2     1cec6fdccdce4740a1d91408aae90ffd       1     None    TW2345_CFD   
3     1cec6fdccdce4740a1d91408aae90ffd       1     None    TW2395_CFD   
4     1cec6fdccdce4740a1d91408aae90ffd       1  SINBFA1         SGAWX   
...                                ...     ...      ...           ...   
3307  622673c88ea7499594d6f227f0eb835d       1  CHNFMX1      CN300567   
3308  622673c88ea7499594d6f227f0eb835d       1  HKGH8X1        HK2269   
3309  622673c88ea7499594d6f227f0eb835d       1  CHNCGX1      CN600572   
3310  622673c88ea7499594d6f227f0eb835d       1  CHNF3U1      CN603105   
3311  622673c88ea7499594d6f227f0eb835d       1  HKGIPA1        HK2057   

                                asset_name  \
0                                            
1         360 SECURITY TECHNOLO

In [24]:
query = '''
    SELECT 
          t1.parent_node
          ,SUM(t1.net_portfolio_risk_contribution) AS total_net_portfolio_risk
          ,t2.client_name
          ,t2.base_currency
          ,t2.risk_model
          ,t2.analysis_date
          ,t2.portfolio
          ,t2.benchmark
    FROM (
        SELECT *
        FROM acq_msci_barra.risk_report_risk_attribution_data
        WHERE NOT (parent_node = '/Total/Local Excess/Residual' AND risk_source <> 'Specific')
          AND NOT (parent_node = '/Total/Local Excess' AND risk_source <> 'Market Timing')
    ) AS t1
    LEFT JOIN acq_msci_barra.risk_report_risk_attribution_header AS t2
    ON t1.report_id = t2.report_id
    WHERE t2.base_currency = 'USD'
        AND t2.analysis_date >= '2023-06-01'
        AND t2.risk_model = 'ASE2S'
        AND t2.portfolio = 'SGADF_EOD_batch'
        AND t2.benchmark = 'Cash'
        AND t2.client_name = 'Guotai Junan International Asset Management (Singapore) Pte Limited'
    GROUP BY 
        t2.client_name
        ,t2.base_currency
        ,t2.risk_model
        ,t2.portfolio
        ,t2.benchmark
        ,t2.analysis_date
        ,t1.parent_node
'''

df = pd.read_sql(query, conn)
df = df.sort_values('analysis_date')
df.head()

,parent_node,total_net_portfolio_risk,client_name,base_currency,risk_model,analysis_date,portfolio,benchmark
49,/Total/Local Excess/Residual/Common Factor,0.413606,Guotai Junan International Asset Management (S...,USD,ASE2S,2023-09-11,SGADF_EOD_batch,Cash
2,/Total/Currency,0.001091,Guotai Junan International Asset Management (S...,USD,ASE2S,2023-09-11,SGADF_EOD_batch,Cash
44,/,2.366058,Guotai Junan International Asset Management (S...,USD,ASE2S,2023-09-11,SGADF_EOD_batch,Cash
27,/Total/Local Excess/Residual/Common Factor/Cou...,0.048387,Guotai Junan International Asset Management (S...,USD,ASE2S,2023-09-11,SGADF_EOD_batch,Cash
102,/Total/Local Excess,0.026607,Guotai Junan International Asset Management (S...,USD,ASE2S,2023-09-11,SGADF_EOD_batch,Cash


In [25]:
df['parent_node'].unique()

array(['/Total/Local Excess/Residual/Common Factor', '/Total/Currency',
       '/', '/Total/Local Excess/Residual/Common Factor/Country',
       '/Total/Local Excess', '/Total', '/Total/Local Excess/Residual',
       '/Total/Local Excess/Residual/Common Factor/Risk Indices',
       '/Total/Local Excess/Residual/Common Factor/Industry',
       '/Total/Local Excess/Residual/Common Factor/Asia Equity'],
      dtype=object)

In [26]:
categories_map = {
    '/Total/Local Excess/Residual/Common Factor/Industry': 'Industry',
    '/Total/Currency': 'Currency',
    '/Total/Local Excess/Residual/Common Factor/Risk Indices': 'Risk Indices',
    '/Total/Local Excess/Residual': 'Specific',
    '/Total/Local Excess/Residual/Common Factor/Country': 'Country',
    '/Total/Local Excess': 'Market Timing',
    '/': 'Ex-Ante Risk'
}

In [27]:
df1 = df.copy()
df1
# df1.to_json("data1.json", orient="columns")

,parent_node,total_net_portfolio_risk,client_name,base_currency,risk_model,analysis_date,portfolio,benchmark
49,/Total/Local Excess/Residual/Common Factor,0.413606,Guotai Junan International Asset Management (S...,USD,ASE2S,2023-09-11,SGADF_EOD_batch,Cash
2,/Total/Currency,0.001091,Guotai Junan International Asset Management (S...,USD,ASE2S,2023-09-11,SGADF_EOD_batch,Cash
44,/,2.366058,Guotai Junan International Asset Management (S...,USD,ASE2S,2023-09-11,SGADF_EOD_batch,Cash
27,/Total/Local Excess/Residual/Common Factor/Cou...,0.048387,Guotai Junan International Asset Management (S...,USD,ASE2S,2023-09-11,SGADF_EOD_batch,Cash
102,/Total/Local Excess,0.026607,Guotai Junan International Asset Management (S...,USD,ASE2S,2023-09-11,SGADF_EOD_batch,Cash
...,...,...,...,...,...,...,...,...
41,/Total/Local Excess/Residual/Common Factor,0.435589,Guotai Junan International Asset Management (S...,USD,ASE2S,2023-09-25,SGADF_EOD_batch,Cash
51,/Total/Local Excess/Residual/Common Factor/Ris...,0.197245,Guotai Junan International Asset Management (S...,USD,ASE2S,2023-09-25,SGADF_EOD_batch,Cash
74,/Total,2.461617,Guotai Junan International Asset Management (S...,USD,ASE2S,2023-09-25,SGADF_EOD_batch,Cash
100,/Total/Local Excess/Residual/Common Factor/Cou...,0.039020,Guotai Junan International Asset Management (S...,USD,ASE2S,2023-09-25,SGADF_EOD_batch,Cash


In [13]:
df['parent_node'] = df['parent_node'].replace(categories_map)
df = df[df['parent_node'].isin(list(categories_map.values()))]
df['analysis_date'] = pd.to_datetime(df['analysis_date'])
# df = df.sort_values(by=['parent_node', 'analysis_date'])
df = df[df['analysis_date'].dt.month == 6] 
# Generate business days for the x-axis
df['analysis_date'] = df['analysis_date'].dt.strftime('%y-%m-%d')

colors = ['#99FF99', '#A0A0A0', '#FFFF33', '#3399FF', '#990000', '#001933']
df.to_json("bar_data_rec.json", orient="records")

In [28]:
import plotly.express as px
import plotly.graph_objects as go

df['parent_node'] = df['parent_node'].replace(categories_map)
df = df[df['parent_node'].isin(list(categories_map.values()))]
df['analysis_date'] = pd.to_datetime(df['analysis_date'])
df = df.sort_values(by=['parent_node', 'analysis_date'])
df = df[df['analysis_date'].dt.month == 9] 
# Generate business days for the x-axis

print(df['analysis_date'].min())
print(df['analysis_date'].max())
business_days = pd.date_range(start=df['analysis_date'].min(), end=df['analysis_date'].max(), freq='B')
# df['analysis_date'] = df['analysis_date'].dt.strftime('%y-%m-%d')

colors = ['#99FF99', '#A0A0A0', '#FFFF33', '#3399FF', '#990000', '#001933']
bar_df = df[~(df['parent_node'] == 'Ex-Ante Risk')]
# bar_df_sorted = bar_df.sort_values(by='parent_node')
fig = px.bar(bar_df, x='analysis_date', y='total_net_portfolio_risk', color='parent_node',
             title='Stacked Bar Chart of Net Portfolio Risk by Parent Node',
             labels={'parent_node': 'Parent Node', 'total_net_portfolio_risk': 'Total Net Portfolio Risk'},
             height=600,
             color_discrete_sequence=colors)

fig.update_layout(barmode='stack', width=1000)
fig.update_yaxes(range=[-1, 4], tickangle=0)
desired_order = ['Industry', 'Risk Indices', 'Market Timing', 'Currency', 'Specific', 'Country']
fig.update_xaxes(tickvals=business_days, ticktext=business_days.strftime('%Y-%m-%d'), tickangle=90, type='category', categoryorder= 'array', categoryarray=desired_order)

ex_ante_risk_data = df[df['parent_node'] == 'Ex-Ante Risk']
fig.add_trace(go.Scatter(
    x=ex_ante_risk_data['analysis_date'],
    y=ex_ante_risk_data['total_net_portfolio_risk'],
    mode='markers+lines',
    name='Ex-ante risk',
    marker=dict(
        color='#004C99'
    )
))

fig.show()

2023-09-11 00:00:00
2023-09-25 00:00:00


In [29]:
df.head()

,parent_node,total_net_portfolio_risk,client_name,base_currency,risk_model,analysis_date,portfolio,benchmark
27,Country,0.048387,Guotai Junan International Asset Management (S...,USD,ASE2S,2023-09-11,SGADF_EOD_batch,Cash
54,Country,0.051183,Guotai Junan International Asset Management (S...,USD,ASE2S,2023-09-12,SGADF_EOD_batch,Cash
43,Country,0.077454,Guotai Junan International Asset Management (S...,USD,ASE2S,2023-09-13,SGADF_EOD_batch,Cash
32,Country,0.069696,Guotai Junan International Asset Management (S...,USD,ASE2S,2023-09-14,SGADF_EOD_batch,Cash
103,Country,0.040497,Guotai Junan International Asset Management (S...,USD,ASE2S,2023-09-15,SGADF_EOD_batch,Cash


In [32]:
query = '''
    SELECT t1.factor
            ,t1.parent_node
            ,SUM(t1.portfolio_net_exposure) AS portfolio_net_exposure
            ,SUM(t1.portfolio_long_exposure) AS portfolio_long_exposure
            ,SUM(t1.portfolio_short_exposure) AS portfolio_short_exposure
            ,SUM(t1.active_risk_pctg_contribution) AS active_risk_pctg_contribution
          ,t2.client_name
          ,t2.base_currency
          ,t2.risk_model
          ,t2.analysis_date
          ,t2.portfolio
          ,t2.benchmark
    FROM (
        SELECT *
        FROM acq_msci_barra.risk_report_factor_risk_data
        WHERE parent_node IN ('/Industry', '/Risk Indices')
    ) AS t1
    LEFT JOIN acq_msci_barra.risk_report_factor_risk_header AS t2
    ON t1.report_id = t2.report_id
    WHERE t2.base_currency = 'USD'
        AND t2.analysis_date = '2023-09-01'
        AND t2.risk_model = 'ASE2S'
        AND t2.portfolio = 'SGADF_EOD_batch'
        AND t2.benchmark = 'Cash'
        AND t2.client_name = 'Guotai Junan International Asset Management (Singapore) Pte Limited'
    GROUP BY 
        t2.client_name
        ,t2.base_currency
        ,t2.risk_model
        ,t2.portfolio
        ,t2.benchmark
        ,t2.analysis_date
        ,t1.factor
        ,t1.parent_node
'''

df = pd.read_sql(query, conn)
df.head()

,factor,parent_node,portfolio_net_exposure,portfolio_long_exposure,portfolio_short_exposure,active_risk_pctg_contribution,client_name,base_currency,risk_model,analysis_date,portfolio,benchmark


In [ ]:
df = df[df['parent_node'] == '/Risk Indices']
df = df[['factor', 'portfolio_net_exposure', 'portfolio_long_exposure',
       'portfolio_short_exposure', 'active_risk_pctg_contribution']]

top_10 = df.nlargest(10, 'portfolio_net_exposure')
top_10
pivot_df = df.pivot_table(index='factor', values=['portfolio_net_exposure', 'portfolio_long_exposure',
                                                          'portfolio_short_exposure', 'active_risk_pctg_contribution'])
pivot_df = pivot_df.reset_index()

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(rows=1, cols=2, shared_yaxes=True,
                    subplot_titles=['Exposure', 'Risk'])

# Add bar traces to each subplot
for col, title in zip(['portfolio_net_exposure', 'portfolio_long_exposure', 'portfolio_short_exposure'],
                      ['Portfolio Net Exposure', 'Portfolio Long Exposure', 'Portfolio Short Exposure']):
    fig.add_trace(go.Bar(x=top_10[col], y=top_10['factor'], orientation='h', name=title), row=1, col=1)

for col, title in zip(['active_risk_pctg_contribution'],
                      ['Risk']):
    fig.add_trace(go.Bar(x=top_10[col], y=top_10['factor'], orientation='h', name=title), row=1, col=2)
    
fig.update_layout(barmode='stack')
fig.update_layout(barmode='relative',  # To stack the bars,
                  yaxis=dict(categoryorder='total ascending'))  # To maintain the order of factors
fig.update_layout(height=600, width=1000, title="Bar Chart of Factor Data")

# Show the plot
fig.show()

NameError: name 'top_10' is not defined

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Assuming you have already executed the SQL query and have the DataFrame 'df'

def create_bar_chart(value, ax):
    sns.barplot(x=[value], y=['Risk'], ax=ax)
    ax.set_xlabel('')
    ax.set_ylabel('')
    ax.yaxis.set_visible(False)  # Hide y-axis labels
    ax.spines['top'].set_visible(False)  # Hide top border
    ax.spines['right'].set_visible(False)  # Hide right border
    ax.spines['bottom'].set_visible(False)  # Hide bottom border
    ax.figure.set_size_inches(10, 5)
    ax.xaxis.set_visible(False)
    ax.set_xlim(0, 0.001)

def create_text_value(value, ax):
    ax.text(0.5, 0.5, f'{value:.2f}', va='center', ha='center',
            fontsize=12, fontweight='bold')
    ax.axis('off')


def create_factor_label(label, ax):
    ax.text(0.5, 0.5, label, va='center', ha='center',
            fontsize=12, fontweight='bold')
    ax.axis('off')
    

# Filter and process data
df_filtered = df[df['parent_node'] == '/Industry']
top_10 = df_filtered.nlargest(10, 'portfolio_net_exposure')

# Create the combined chart
fig, axes = plt.subplots(nrows=10, ncols=3, figsize=(12, 30))

for idx, factor_row in enumerate(top_10.itertuples(), start=0):
    factor = factor_row.factor
    risk_value = factor_row.active_risk_pctg_contribution
    

    axes[0, 1].set_title('Value')
    axes[0, 0].set_title('Factor')
    axes[0, 2].set_title('Risk Contribution')
    create_factor_label(factor, axes[idx, 0])
    create_text_value(factor_row.portfolio_net_exposure, axes[idx, 1])
    create_bar_chart(risk_value, axes[idx, 2])

plt.tight_layout()
plt.show()